In [1]:
import pandas as pd  
from pandas import Series, DataFrame 
import uproot 
from scipy import stats
from scipy.optimize import curve_fit
from scipy.special import comb
from scipy.stats import chi2
from scipy.special import comb
from scipy.optimize import lsq_linear
import sys
from plot_tools import *
from customStats import *
#import tools
import common_tools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
# from selection_cuts import selection_nominal
import mplhep as hep
from sklearn.model_selection import train_test_split
plt.style.use(hep.style.CMS)
plt.rcParams['figure.figsize'] = [10,8]
plt.rcParams['font.size'] = 24
plt.figure()
plt.close()
plt.rcParams.update({'figure.figsize':[10,8]})
plt.rcParams.update({'font.size':24})
import tensorflow as tf
import math
import zfit
from zfit import z
import xgboost as xgb
from scipy.interpolate import make_interp_spline
# from loadCutXGB import load_and_cutXGBclfs
from scipy.special import comb
from scipy.optimize import lsq_linear
zfit.settings.set_verbosity(0)
import json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Oculta los mensajes de INFO y WARNING
from PDFs import *

2026-02-21 11:39:04.914546: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-21 11:39:05.087182: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-21 11:39:05.672860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ghcp/miniconda3/envs/haza_wokr_env/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress th

# FUNCTIONS FOR CALCULATING EFFICIENCY

In [2]:

def bernstein_1d(n, k, t):
    """Bernstein base polynomial B_{n,k}(t) on t in [0,1]."""
    return comb(n, k) * (t**k) * ((1.0 - t)**(n - k))


def bernstein2d_matrix(nx, ny, x, y):
    """
    Build the design matrix for Bernstein2D basis.
    Input: x, y in [-1,1] (arrays)
    Output: B matrix of size (Npoints, (nx+1)*(ny+1))
    """
    # map [-1,1] -> [0,1]
    tx = 0.5*(x + 1.0)
    ty = 0.5*(y + 1.0)

    B_list = []
    for i in range(nx+1):
        for j in range(ny+1):
            B_list.append(bernstein_1d(nx, i, tx) * bernstein_1d(ny, j, ty))
    B = np.vstack(B_list).T   # shape (Npoints, Ncoef)
    return B


# ======================================================
# 2) Fit Bernstein2D to a 2D efficiency map
# ======================================================
def fit_bernstein2d( xcenters, ycenters, eff2d, ngen2d, nx=8, ny=8, min_counts_mask=None,reg_lambda=1e-10,):
    """
    Fits a Bernstein2D polynomial to a 2D efficiency map using least squares + Tikhonov reg.

    Inputs:
        xcenters, ycenters   1D arrays (bin centers)
        eff2d                2D array with efficiency values (NaNs allowed)
        nx, ny               polynomial orders
        min_counts_mask      Boolean 2D mask (valid bins: True)
        reg_lambda           regularization parameter

    Returns:
        coef                 fitted coefficients
        eff_model            modeled efficiency on the grid

    
    Weighted fit of a Bernstein2D polynomial to a 2D efficiency map.

    The weights are derived from binomial uncertainties:
        sigma^2 = eff * (1 - eff) / ngen
    """

    XX, YY = np.meshgrid(xcenters, ycenters, indexing="ij")
    xflat = XX.ravel()
    yflat = YY.ravel()
    eff_flat = eff2d.ravel()
    ngen_flat = ngen2d.ravel()

    # Valid bins
    if min_counts_mask is None: use = (~np.isnan(eff_flat)) & (ngen_flat > 0)
    else:
        use = (min_counts_mask.ravel() & ~np.isnan(eff_flat)& (ngen_flat > 0))

    x_use = xflat[use]
    y_use = yflat[use]
    eff_use = eff_flat[use]
    ngen_use = ngen_flat[use]

    # Binomial uncertainty
    sigma2 = eff_use * (1.0 - eff_use) / ngen_use
    sigma2 = np.clip(sigma2, 1e-12, None)
    w = 1.0 / np.sqrt(sigma2)
    B = bernstein2d_matrix(nx, ny, x_use, y_use)

    # Apply weights
    Bw = B * w[:, None]
    yw = eff_use * w

    # Regularized weighted least squares
    BTB = Bw.T @ Bw + reg_lambda * np.eye(B.shape[1])
    BTy = Bw.T @ yw
    coef = np.linalg.solve(BTB, BTy)
    Bfull = bernstein2d_matrix(nx, ny, xflat, yflat)
    eff_model_flat = Bfull @ coef
    eff_model = eff_model_flat.reshape(eff2d.shape)
    return coef, eff_model


def build_efficiency_2d(gen_all_x, gen_all_y, gen_fid_x, gen_fid_y, reco_fid_x, reco_fid_y, reco_x, reco_y, 
                        weights_reco=None, nbx=20, nby=20, nxg=8, nyg=8, nxr=8, nyr=8, min_gen=0, reg_acc=1e-4, reg_reco=1e-4):
    xedges = np.linspace(-1, 1, nbx + 1)
    yedges = np.linspace(-1, 1, nby + 1)
    xcenters = 0.5 * (xedges[:-1] + xedges[1:])
    ycenters = 0.5 * (yedges[:-1] + yedges[1:])

    # Pesos 
    if weights_reco is None:
        weights_reco = np.ones(len(reco_x))

    # Histograms
    gen_allH, _, _ = np.histogram2d(gen_all_x, gen_all_y, bins=[xedges, yedges])
    gen_fidH, _, _ = np.histogram2d(gen_fid_x, gen_fid_y, bins=[xedges, yedges])
    # Denominador Reco
    reco_fidH, _, _ = np.histogram2d(reco_fid_x, reco_fid_y, bins=[xedges, yedges])    
    # Numerador Reco (CON PESOS APLICADOS)
    recoH, _, _ = np.histogram2d(reco_x, reco_y, bins=[xedges, yedges], weights=weights_reco)

    # ============================
    # Acceptance
    # ============================
    mask_gen = gen_allH > min_gen
    acc_gen = np.full_like(gen_allH, np.nan)
    with np.errstate(divide='ignore', invalid='ignore'):
        acc_gen[mask_gen] = gen_fidH[mask_gen] / gen_allH[mask_gen]

    # ============================
    # Reconstruction efficiency
    # ============================
    eff_reco = np.full_like(gen_allH, np.nan)
    valid = mask_gen & (reco_fidH > 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        eff_reco[valid] = recoH[valid] / reco_fidH[valid]
    
    # ============================
    # Bernstein fits
    # ============================
    # Nota: Los fits también deben saber que recoH ahora tiene pesos (float), no solo counts (int).
    # Asegúrate de que fit_bernstein2d maneje arrays de floats en "data_hist" (recoH).
    coef_acc, acc_gen_model = fit_bernstein2d( xcenters, ycenters, acc_gen, gen_allH, nx=nxg, ny=nyg, min_counts_mask=mask_gen, reg_lambda=reg_acc)
    coef_reco, eff_reco_model = fit_bernstein2d(xcenters, ycenters, eff_reco, reco_fidH, nx=nxr, ny=nyr, min_counts_mask=valid, reg_lambda=reg_reco)

    return (xcenters, ycenters, acc_gen, acc_gen_model, coef_acc, eff_reco, eff_reco_model, coef_reco, mask_gen)


def build_efficiency_1d(gen_all, gen_fid, reco_fid, reco, weights_reco=None, nbins=30, n_poly=4, min_gen=10, reg_acc=1e-6, reg_reco=1e-5):

    limit_min, limit_max = -np.pi, np.pi
    edges = np.linspace(limit_min, limit_max, nbins + 1)
    centers = 0.5 * (edges[:-1] + edges[1:])
    centers_norm = 2 * (centers - limit_min) / (limit_max - limit_min) - 1.0

    # Pesos
    if weights_reco is None:
        weights_reco = np.ones(len(reco))
    
    # Histograms
    gen_allH, _ = np.histogram(gen_all, bins=edges)
    gen_fidH, _ = np.histogram(gen_fid, bins=edges)
    reco_fidH, _ = np.histogram(reco_fid, bins=edges)
    
    # Numerador Reco CON PESOS
    recoH, _ = np.histogram(reco, bins=edges, weights=weights_reco)

    # Acceptance
    mask_gen = gen_allH > min_gen
    acc_gen = np.full_like(gen_allH, np.nan, dtype=float)
    with np.errstate(divide='ignore', invalid='ignore'):
        acc_gen[mask_gen] = gen_fidH[mask_gen] / gen_allH[mask_gen]
    coef_acc, acc_model = fit_bernstein1d( centers_norm, acc_gen, gen_allH, n=n_poly, min_counts_mask=mask_gen, reg_lambda=reg_acc)
    
    # Efficiency reco
    eff_reco = np.full_like(gen_allH, np.nan, dtype=float)
    valid_reco = mask_gen & (reco_fidH > 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        eff_reco[valid_reco] = recoH[valid_reco] / reco_fidH[valid_reco]
    coef_reco, eff_reco_model = fit_bernstein1d(centers_norm, eff_reco, reco_fidH, n=n_poly, min_counts_mask=valid_reco, reg_lambda=reg_reco)

    return centers, acc_gen, acc_model, coef_acc, eff_reco, eff_reco_model, coef_reco, mask_gen


def bernstein2d_eval(x, y, model):
    """
    Evaluate a fitted Bernstein2D model.
    """
    nx = model["nx"]
    ny = model["ny"]
    coef = np.asarray(model["coef"])
    tx = 0.5 * (x + 1.0)
    ty = 0.5 * (y + 1.0)
    eff = np.zeros_like(tx, dtype=float)
    idx = 0
    for i in range(nx + 1):
        Bx = bernstein_1d(nx, i, tx)
        for j in range(ny + 1):
            By = bernstein_1d(ny, j, ty)
            eff += coef[idx] * Bx * By
            idx += 1

    return eff


# ======================================================
# Save / Load Bernstein models
# ======================================================
def save_bernstein2d_model(filename, coef, nx, ny):
    model = {"nx": nx, "ny": ny, "coef": coef.tolist(), "x_range": [-1.0, 1.0], "y_range": [-1.0, 1.0]}
    directory = os.path.dirname(filename)
    if directory:
        os.makedirs(directory, exist_ok=True)
    with open(filename, "w") as f:
        json.dump(model, f, indent=2)


def load_bernstein_model(filename):
    with open(filename) as f:
        model = json.load(f)
    return (np.asarray(model["coef"], dtype=np.float64), model["nx"], model["ny"])


# ======================================================
#   plots
# ======================================================
def project_with_errors_x(data2d, mask):
    """Project 2D efficiency to x with diagnostic errors."""
    proj = []
    err = []
    for i in range(data2d.shape[0]):   # x bins
        vals = data2d[i, :][mask[i, :]]
        if len(vals) == 0:
            proj.append(np.nan)
            err.append(np.nan)
        else:
            proj.append(np.mean(vals))
            err.append(np.std(vals, ddof=0) / np.sqrt(len(vals)))
    return np.array(proj), np.array(err)


def project_with_errors_y(data2d, mask):
    """Project 2D efficiency to y with diagnostic errors."""
    proj = []
    err = []
    for j in range(data2d.shape[1]):   # y bins
        vals = data2d[:, j][mask[:, j]]
        if len(vals) == 0:
            proj.append(np.nan)
            err.append(np.nan)
        else:
            proj.append(np.mean(vals))
            err.append(np.std(vals, ddof=0) / np.sqrt(len(vals)))
    return np.array(proj), np.array(err)


# ======================================================
# CMS Style Plotting
# ======================================================
def _plot_cms_style(centers, data, data_err, model, xlabel, title, y_label="Efficiency", ylim=None, path_dir="plots"):
    fig = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(2, 1, height_ratios=[3.5, 1.5], hspace=0.05)
    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1], sharex=ax0)

    # --- AUTO-SCALING LOGIC ---
    if ylim is None:
        # Calculamos el punto más alto considerando el error superior
        # Usamos nanmax para ignorar posibles NaNs
        max_data = np.nanmax(data + data_err) if data_err is not None else np.nanmax(data)
        max_model = np.nanmax(model)
        global_max = max(max_data, max_model)
        
        # Si por alguna razón todo es 0 o NaN, ponemos un default
        if np.isnan(global_max) or global_max <= 0:
            global_max = 1.0
            
        # Definimos el límite: 0 abajo, y Max + 30% arriba para la leyenda/CMS label
        current_ylim = (0.0, global_max * 1.30)
    else:
        current_ylim = ylim
    # --------------------------

    # Plot principal
    ax0.plot(centers, model, '-', color='blue', linewidth=2.5, label="Bernstein Model")
    ax0.errorbar(centers, data, yerr=data_err, fmt='ks', markersize=5, elinewidth=1.5, capsize=2, label="Binned MC")
    ax0.set_ylabel(y_label, fontsize=16)
    ax0.set_title(title, loc='center', fontsize=14, fontweight='medium', y=1.05)
    
    # Aplicamos el límite calculado o el manual
    ax0.set_ylim(current_ylim)

    hep.cms.label(data=False, loc=0, ax=ax0, rlabel="13 TeV", fontname="sans-serif", fontsize=16)
    ax0.legend(frameon=False, fontsize=13, loc='upper right')
    ax0.grid(True, alpha=0.3)
    plt.setp(ax0.get_xticklabels(), visible=False)

    # Pulls (El resto sigue igual)
    with np.errstate(divide='ignore', invalid='ignore'):
        pulls = (data - model) / data_err
    pulls[~np.isfinite(pulls)] = 0.0 

    width = centers[1] - centers[0]
    lower = centers[0] - width/2
    upper = centers[-1] + width/2

    ax1.errorbar(centers, pulls, yerr=1.0, xerr=0, fmt='ks',markersize=4, elinewidth=1.0,capsize=0)          
    ax1.axhline(0, color='black', linewidth=1.0, linestyle='-')
    ax1.axhline(3, color='gray', linestyle=':', linewidth=1, alpha=0.8) 
    ax1.axhline(-3, color='gray', linestyle=':', linewidth=1, alpha=0.8)    
    ax1.fill_between([lower, upper], -3, 3, color='gray', alpha=0.15, label=r'$3\sigma$') 
    ax1.set_xlabel(xlabel, fontsize=16)
    ax1.set_ylabel(r'Pull $(\sigma)$', fontsize=13)
    ax1.set_xlim(lower, upper)
    ax1.set_ylim(-4.9, 4.9)
    ax1.grid(True, alpha=0.3)
    ax0.tick_params(axis='both', which='major', labelsize=14, direction='in', top=True, right=True) 
    ax1.tick_params(axis='both', which='major', labelsize=14, direction='in', top=True, right=True)
    
    # Guardado seguro
    save_path = os.path.join(path_dir, f"{title}.png")
    directory = os.path.dirname(save_path)
    if directory:
        os.makedirs(directory, exist_ok=True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


# ======================================================
# Public Functions (Wrappers)
# ======================================================
def plot_projection_x_with_errors(xc, data2d, model2d, mask, title, ylim=None, path=None):
    """
    Proyección en X (cosThetaL)
    """

    data_proj, data_err = project_with_errors_x(data2d, mask)
    model_proj, _ = project_with_errors_x(model2d, mask)
    if np.nanmean(model_proj) != 0:
        scale = np.nanmean(data_proj) / np.nanmean(model_proj)
    else:
        scale = 1.0
    model_proj_scaled = model_proj * scale
    _plot_cms_style(centers=xc, data=data_proj, data_err=data_err, model=model_proj_scaled, xlabel=r"$\cos\theta_\ell$", title=title, ylim=ylim, path_dir=path)


def plot_projection_y_with_errors(yc, data2d, model2d, mask, title, ylim, path):
    """
    Proyección en Y (cosThetaK)
    """
    data_proj, data_err = project_with_errors_y(data2d, mask)
    model_proj, _ = project_with_errors_y(model2d, mask)
    
    if np.nanmean(model_proj) != 0:
        scale = np.nanmean(data_proj) / np.nanmean(model_proj)
    else:
        scale = 1.0
    model_proj_scaled = model_proj * scale

    _plot_cms_style(centers=yc, data=data_proj, data_err=data_err, model=model_proj_scaled, xlabel=r"$\cos\theta_K$", title=title, ylim=ylim, path_dir=path) 


def select_q2_bin(df, n_bin, cut):
    q2_bins = dict()
    q2_bins = { "bin0":[1.1,23.0],   "bin1":[1.1, 2.0],"bin2": [2.0, 4.0],"bin3":[4.0, 6.0],
                "bin4":[6.0, 7.0],   "bin5":[7.0, 8.0], "bin6": [8.0, 11.0],"bin7":[11.0, 12.5],
                "bin8":[12.5, 15.0], "bin9":[15.0, 17.0], "bin10":[17.0, 23.0]}
    df_ = df[(df[cut]>=q2_bins[n_bin][0]) & (df[cut] <= q2_bins[n_bin][1])].copy()
    return df_



# ======================================================
# PHI  1D Bernstei
# ======================================================

def save_bernstein1d_model(filename, coef, n):
    directory = os.path.dirname(filename)
    if directory:
        os.makedirs(directory, exist_ok=True)
    model = {"n": n, "coef": coef.tolist(), "range": [-np.pi, np.pi]}
    with open(filename, "w") as f:
        json.dump(model, f, indent=2)

def bernstein1d_matrix(n, x):
    """
    Build the design matrix for Bernstein 1D basis.
    Input: x in [-1, 1] (array)
    Output: B matrix of size (Npoints, n+1)
    """
    # map [-1,1] -> [0,1]
    t = 0.5 * (x + 1.0)
    B_list = []
    for i in range(n+1):
        B_list.append(bernstein_1d(n, i, t))
    B = np.vstack(B_list).T

    return B


def fit_bernstein1d(xcenters, eff1d, ngen1d, n=4, min_counts_mask=None, reg_lambda=1e-10):
    """
    Fits a Bernstein 1D polynomial to a 1D efficiency histogram.
    """
    # Filter NaNs and Zero Gen
    if min_counts_mask is None:
        use = (~np.isnan(eff1d)) & (ngen1d > 0)
    else:
        use = min_counts_mask & ~np.isnan(eff1d) & (ngen1d > 0)
        
    x_use = xcenters[use]
    eff_use = eff1d[use]
    ngen_use = ngen1d[use]
    # Binomial uncertainty weights
    sigma2 = eff_use * (1.0 - eff_use) / ngen_use
    sigma2 = np.clip(sigma2, 1e-12, None)
    w = 1.0 / np.sqrt(sigma2)
    
    B = bernstein1d_matrix(n, x_use)
    Bw = B * w[:, None]
    yw = eff_use * w
    BTB = Bw.T @ Bw + reg_lambda * np.eye(B.shape[1])
    BTy = Bw.T @ yw
    coef = np.linalg.solve(BTB, BTy)
    Bfull = bernstein1d_matrix(n, xcenters)
    eff_model = Bfull @ coef
    
    return coef, eff_model

def load_bernstein1d_model(filename):
    """Carga un modelo de Bernstein 1D desde un JSON."""

    with open(filename, 'r') as f:
        data = json.load(f)
    coef = np.asarray(data["coef"], dtype=np.float64)
    if "degree" in data:
        degree = data["degree"]
    else:
        degree = len(coef) - 1
        
    return coef, degree


def plot_1d_result(centers, data, model, mask, title, ylim, path):
    valid = mask
    dummy_err = np.zeros_like(data)
    dummy_err[valid] = 0.05 * data[valid]     
    _plot_cms_style(centers[valid], data[valid], dummy_err[valid], model[valid], xlabel=r"$\phi$", title=title,ylim=ylim, path_dir=path)


def run_fit(model, data):
    nll = zfit.loss.UnbinnedNLL(model=model, data=data)
    minimizer = zfit.minimize.Minuit()
    result = minimizer.minimize(nll)
    err = None
    try:
        err, _ = result.errors(name="minos", method="minuit_minos", cl=0.682)
    except Exception as e:
        print("MINOS failed:", e)
    return result, err

# =====================================================
# CODE FOR FIT INCLUDING EFFICIENCY
# =====================================================

def tf_bernstein_basis_vectorized(n, t):
    M = tf.shape(t)[0]
    k = tf.range(n + 1, dtype=tf.float64)
    n_float = tf.cast(n, tf.float64)
    log_binom = tf.math.lgamma(n_float + 1.0) - tf.math.lgamma(k + 1.0) - tf.math.lgamma(n_float - k + 1.0)
    binom = tf.exp(log_binom)
    t_col = tf.expand_dims(t, -1) 
    k_row = tf.expand_dims(k, 0)
    term1 = tf.pow(t_col, k_row)
    term2 = tf.pow(1.0 - t_col, n_float - k_row)
    basis = binom * term1 * term2 
    return basis


class Efficiency_Bernstein_Factorized(zfit.pdf.BasePDF):
    def __init__(self, obs,coef_acc_2d, coef_acc_phi, nx_acc, ny_acc, n_phi_acc, coef_reco_2d, coef_reco_phi, nx_reco, ny_reco, n_phi_reco,
                 name="Full_Efficiency_Model"):
        """
        Modelo Completo: Aceptancia * Eficiencia de Reconstrucción.
        Cada parte factorizada en 2D(cosL, cosK) * 1D(phi).
        """
        params = {
            'c_acc_2d': zfit.Parameter(f"c_a2d_{name}", tf.cast(coef_acc_2d, tf.float64), floating=False),
            'c_acc_phi': zfit.Parameter(f"c_aphi_{name}", tf.cast(coef_acc_phi, tf.float64), floating=False),
            'c_reco_2d': zfit.Parameter(f"c_r2d_{name}", tf.cast(coef_reco_2d, tf.float64), floating=False),
            'c_reco_phi': zfit.Parameter(f"c_rphi_{name}", tf.cast(coef_reco_phi, tf.float64), floating=False),
        }
        
        # Guardamos los grados de los polinomios
        self.nx_acc, self.ny_acc = nx_acc, ny_acc
        self.n_phi_acc = n_phi_acc
        self.nx_reco, self.ny_reco = nx_reco, ny_reco
        self.n_phi_reco = n_phi_reco
        
        super().__init__(obs, params, name=name)

    def _unnormalized_pdf(self, x):
        vars_list = z.unstack_x(x)
        cos_l, cos_k, phi = vars_list[0], vars_list[1], vars_list[2]

        # [-1, 1] -> [0, 1] y [-pi, pi] -> [0, 1]
        tx = 0.5 * (cos_l + 1.0)
        ty = 0.5 * (cos_k + 1.0)
        t_phi = (phi + np.pi) / (2.0 * np.pi)
        # ======================================================
        # ACEPTANCIA
        # ======================================================
        # Bases
        Bx_acc = tf_bernstein_basis_vectorized(self.nx_acc, tx)
        By_acc = tf_bernstein_basis_vectorized(self.ny_acc, ty)
        Bphi_acc = tf_bernstein_basis_vectorized(self.n_phi_acc, t_phi)
        
        # 2D part
        c_acc_2d_mat = tf.reshape(self.params['c_acc_2d'], (self.nx_acc + 1, self.ny_acc + 1))
        acc_2d = tf.einsum('mi,mj,ij->m', Bx_acc, By_acc, c_acc_2d_mat)
        # 1D part
        acc_phi = tf.einsum('mk,k->m', Bphi_acc, self.params['c_acc_phi'])
        # Total Acceptance
        total_acc = acc_2d * acc_phi

        # ======================================================
        # EFICIENCIA DE RECONSTRUCCIÓN
        # ======================================================
        # Bases
        Bx_reco = tf_bernstein_basis_vectorized(self.nx_reco, tx)
        By_reco = tf_bernstein_basis_vectorized(self.ny_reco, ty)
        Bphi_reco = tf_bernstein_basis_vectorized(self.n_phi_reco, t_phi)
        
        # 2D part
        c_reco_2d_mat = tf.reshape(self.params['c_reco_2d'], (self.nx_reco + 1, self.ny_reco + 1))
        reco_2d = tf.einsum('mi,mj,ij->m', Bx_reco, By_reco, c_reco_2d_mat)
        # 1D part
        reco_phi = tf.einsum('mk,k->m', Bphi_reco, self.params['c_reco_phi'])
        # Total Reco Efficiency
        total_reco = reco_2d * reco_phi

        # ======================================================
        # EFICIENCIA FINAL
        # ======================================================
        return tf.maximum(total_acc * total_reco, 1e-15)


def save_fit_results(result, bin_n, base_dir="fit_results", name="fit_results"):
    """
    Guarda resultados buscando errores con nombres personalizados ('minos') 
    o por defecto ('minuit_minos', 'minuit_hesse').
    """
    
    output_folder = os.path.join(base_dir, f"{bin_n}")
    os.makedirs(output_folder, exist_ok=True)
    output_file = os.path.join(output_folder, f"{name}.json")
    
    params_dict = {}
    
    for p in result.params:
        val = result.params[p]['value']
        p_data = result.params[p] 

        lower_err = 0.0
        upper_err = 0.0
        sym_err = 0.0
        error_type = "none"

        # busca minos
        if 'minos' in p_data:
            err_data = p_data['minos']
            lower_err = err_data.get('lower', 0.0)
            upper_err = err_data.get('upper', 0.0)
            sym_err = (abs(lower_err) + abs(upper_err)) / 2.0
            error_type = "minos (custom)"

        # busca minuit_minos
        elif 'minuit_minos' in p_data:
            err_data = p_data['minuit_minos']
            lower_err = err_data.get('lower', 0.0)
            upper_err = err_data.get('upper', 0.0)
            sym_err = (abs(lower_err) + abs(upper_err)) / 2.0
            error_type = "minos (default)"
            
        # busca Hesse
        elif 'minuit_hesse' in p_data:
            err_data = p_data['minuit_hesse']
            sym_err = err_data.get('error', -999.0)
            lower_err = -sym_err
            upper_err = sym_err
            error_type = "hesse"
            

        params_dict[p.name] = {'value': float(val), 'error': float(sym_err), 'error_low': float(lower_err), 'error_up': float(upper_err), 'error_source': error_type}


    cov_matrix = result.covariance()
    cov_list = np.array(cov_matrix).tolist()
    data_to_save = {'bin_index': str(bin_n), 'valid': bool(result.valid), 'converged': bool(result.converged), 'fmin': float(result.fmin), 'status': result.status,'parameters': params_dict,'covariance': cov_list}
    
    with open(output_file, 'w') as f:
        json.dump(data_to_save, f, indent=4)
        
    print(f"[CheckPoint] Resultados guardados en: {output_file}")
    return output_file


def save_correlation_matrix(result, params_list, bin_name, out_dir="plots/correlations"):

    os.makedirs(out_dir, exist_ok=True)    
    corr_matrix_raw = result.correlation()    
    zfit_params = list(result.params.keys())
    n_params = len(params_list)
    corr_matrix = np.zeros((n_params, n_params))    
    param_names = [p.name.split('_')[0] for p in params_list]    
    for i, p1 in enumerate(params_list):
        for j, p2 in enumerate(params_list):
            idx1 = zfit_params.index(p1)
            idx2 = zfit_params.index(p2)
            corr_matrix[i, j] = corr_matrix_raw[idx1, idx2]
                
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, xticklabels=param_names, yticklabels=param_names, fmt=".2f", square=True, cbar_kws={"shrink": .8})
    plt.title(f"Matriz de Correlación - {bin_name}", fontsize=14, pad=15)
    plt.xticks(rotation=45)
    plt.tight_layout()
    filepath = os.path.join(out_dir, f"corr_matrix_{bin_name}.png")
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close() 

    


#  CODE FOR EFFICIENCY CALCULATION

In [3]:
# ERA="2022"
# YEAR='20'+ ERA.split("20")[1]
# path = f'/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/efficiencies/Bstomumuphi_{YEAR}/'
# qsBinning=[]
# #if args.decay == 'nonResonant':
# qsBinning = ["bin"+str(i+1) for i in range(8)]
# qsBinning.remove("bin4")
# qsBinning.remove("bin6")
# #elif args.decay == 'ResonantJpsi':
# #    qsBinning = ["bin4"]
# #elif args.decay == 'ResonantPsi':
# #    qsBinning = ["bin6"]

In [4]:
import uproot
import pandas as pd

# --- RUTAS DE ARCHIVOS ---
f_gen = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/efficiencies/datasets/GenLevel_Angular_Merged.root"
f_gen_filtered = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/efficiencies/datasets/GenLevel_Angular_Merged_Filtered.root"
f_reco_gen = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/efficiencies/datasets/RecoGenV2_Angular_Merged.root"  
x_gboost_cut = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/BdtoK0smumu-20251110T171511Z-1-001/MyReweiting/ResultsB0_2022/AntiRadVeto_MC_NoRes_2022_Era1_v0_XGBoost_fom_cut_BDT.root"

vars_gen_to_load = ["gen_cosThetaK", "gen_cosThetaL", "gen_phi", "q2Gen"]
vars_reco_to_load = ["CosThetaK_best", "CosThetaL_best", "Phi_best", "massJ"] 
vars_xgboost_to_load = ["CosThetaK", "CosThetaL", "Phi", "massB_test", "massJ", "TotalWeight"] 

# --- CARGA DE DATOS ---
#Gen NO filt
genNFtr = uproot.open(f_gen)['ntuple'].arrays(vars_gen_to_load, library='pd')
print(f"1. Gen Non-Filtered (genNFtr) cargado: {len(genNFtr)} eventos")
# Gen Filtered
genFtr = uproot.open(f_gen_filtered)['ntuple'].arrays(vars_gen_to_load, library='pd')
print(f"2. Gen Filtered (genFtr) cargado: {len(genFtr)} eventos")
# Reco Gen Level
recoGen = uproot.open(f_reco_gen)['ntuple'].arrays(vars_reco_to_load, library='pd')
print(f"3. Reco Gen Level Denom (recoGen) cargado: {len(recoGen)} eventos")
# Final selection 
recoFtr = uproot.open(x_gboost_cut)['treeBd'].arrays(vars_xgboost_to_load, library='pd')
print(f"4. Reco Final (recoFtr) cargado: {len(recoFtr)} eventos")



1. Gen Non-Filtered (genNFtr) cargado: 11589148 eventos
2. Gen Filtered (genFtr) cargado: 307688 eventos
3. Reco Gen Level Denom (recoGen) cargado: 6298017 eventos
4. Reco Final (recoFtr) cargado: 900424 eventos


In [5]:

recoFtr["q2"] = recoFtr["massJ"]**2 
recoGen["q2Gen"] = recoGen["massJ"]**2  

GenNFlt = genNFtr.copy()     
GenFlt  = genFtr.copy()       

RecoGenFlt = recoGen.copy()             
mask_mass = (recoFtr["massB_test"] > 5.0) & (recoFtr["massB_test"] < 5.6)
Reco = recoFtr[mask_mass].copy()
#2
eff_Gen, obs_Gen = train_test_split(GenNFlt, test_size=0.005, random_state=2)
eff_GenFtr, obs_GenFtr = train_test_split(GenFlt, test_size=0.1, random_state=2)
eff_RecoGenFtr, obs_RecoGenFtr = train_test_split(RecoGenFlt, test_size=0.3, random_state=2)
eff_RecoFtr, obs_RecoFtr = train_test_split(Reco, test_size=0.1, random_state=2)

a1 = np.array(obs_Gen["gen_cosThetaL"])
a2 = np.array(obs_Gen["gen_cosThetaK"])
a3 = np.array(obs_Gen["gen_phi"])

angles = np.array([a1, a2, a3])
valid_observations_mask = ~np.isnan(angles).any(axis=0)
filtered_data = angles[:, valid_observations_mask]

In [ ]:

# corr_matrix = np.corrcoef(filtered_data)
# print("Matriz de correlación (Gen Level):")
# print(f"{'':>15} {'cos(theta_L)':>12} {'cos(theta_K)':>12} {'phi':>12}")
# print(f"{'cos(theta_L)':>15} {corr_matrix[0,0]:12.4f} {corr_matrix[0,1]:12.4f} {corr_matrix[0,2]:12.4f}")
# print(f"{'cos(theta_K)':>15} {corr_matrix[1,0]:12.4f} {corr_matrix[1,1]:12.4f} {corr_matrix[1,2]:12.4f}")
# print(f"{'phi':>15}          {corr_matrix[2,0]:12.4f} {corr_matrix[2,1]:12.4f} {corr_matrix[2,2]:12.4f}")

# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.set(style="whitegrid")
# plt.figure(figsize=(8, 6)) 
# labels = [r"$\cos(\theta_{\ell})$", r"$\cos(\theta_K)$", r"$\phi$"]

# sns.heatmap(corr_matrix, annot=True, fmt=".5f", cmap="coolwarm", vmin=-1, vmax=1, xticklabels=labels, yticklabels=labels)
# plt.title(f"Correlation Matrix - Gen Level (2022)")
# plt.tight_layout()
# plt.show()

In [ ]:
nbx=20
nby=20
nx_gen=4 
ny_gen=4 
nx_rec=4 
ny_rec=4
ylim_ck_a=(0.0, 0.040)
ylim_cl_a=(0.0, 0.040)
ylim_ck_r=(0.0, 0.30)
ylim_cl_r=(0.2, 0.450)
ylim_ck_t=(0.0, 0.008)
ylim_cl_t=(0.0, 0.008)
ylim_p_a=(0.0, 0.06)
ylim_p_r=(0.0, 0.25)
ylim_p_t=(0.0, 0.006)

bin_configs = {
    "bin1":  {"q2_range": [1.1, 2.0],   "nbx": 12, "nby": 12, "nx_gen": 5, "ny_gen": 5, "nx_rec": 5, "ny_rec": 5},
    "bin2":  {"q2_range": [2.0, 4.0],   "nbx": nbx, "nby": nby, "nx_gen": nx_gen, "ny_gen": ny_gen, "nx_rec": nx_rec, "ny_rec": ny_rec},
    "bin3":  {"q2_range": [4.0, 6.0],   "nbx": nbx, "nby": nby, "nx_gen": nx_gen, "ny_gen": ny_gen, "nx_rec": nx_rec, "ny_rec": ny_rec},
    "bin4":  {"q2_range": [6.0, 7.0],   "nbx": nbx, "nby": nby, "nx_gen": nx_gen, "ny_gen": ny_gen, "nx_rec": nx_rec, "ny_rec": ny_rec},
    "bin5":  {"q2_range": [7.0, 8.0],   "nbx": nbx, "nby": nby, "nx_gen": nx_gen, "ny_gen": ny_gen, "nx_rec": nx_rec, "ny_rec": ny_rec},
    "bin7":  {"q2_range": [11.0, 12.5], "nbx": nbx, "nby": nby, "nx_gen": nx_gen, "ny_gen": ny_gen, "nx_rec": nx_rec, "ny_rec": ny_rec},
    "bin9":  {"q2_range": [15.0, 17.0], "nbx": nbx, "nby": nby, "nx_gen": nx_gen, "ny_gen": ny_gen, "nx_rec": nx_rec, "ny_rec": ny_rec},
    "bin10": {"q2_range": [17.0, 23.0], "nbx": nbx, "nby": nby, "nx_gen": nx_gen, "ny_gen": ny_gen, "nx_rec": nx_rec, "ny_rec": ny_rec}
}


for binN, config in bin_configs.items():
    print(f"\n=== Procesando {binN} ===")
    nbx=config["nbx"]
    nby=config["nby"]
    nx_gen=config["nx_gen"]
    ny_gen=config["ny_gen"]
    nx_rec=config["nx_rec"]
    ny_rec=config["ny_rec"]

    eff_Gen_q2 =        select_q2_bin(eff_Gen, binN, "q2Gen")
    eff_GenFtr_q2 =     select_q2_bin(eff_GenFtr, binN, "q2Gen")
    eff_RecoGenFtr_q2 = select_q2_bin(eff_RecoGenFtr, binN, "q2Gen")
    eff_RecoFtr_q2 =    select_q2_bin(eff_RecoFtr, binN, "q2") 

    gen_x = eff_Gen_q2["gen_cosThetaL"].values  
    gen_y = eff_Gen_q2["gen_cosThetaK"].values      
    genFid_x = eff_GenFtr_q2["gen_cosThetaL"].values 
    genFid_y = eff_GenFtr_q2["gen_cosThetaK"].values
    recoFid_x = eff_RecoGenFtr_q2["CosThetaL_best"].values 
    recoFid_y = eff_RecoGenFtr_q2["CosThetaK_best"].values
    reco_x = eff_RecoFtr_q2["CosThetaL"].values
    reco_y = eff_RecoFtr_q2["CosThetaK"].values
    reco_w = eff_RecoFtr_q2["TotalWeight"].values  

    xcenters, ycenters, acc_gen, acc_gen_model, coef_acc, eff_reco, eff_reco_model, coef_reco, mask_gen = build_efficiency_2d( 
        gen_x, gen_y, genFid_x, genFid_y, recoFid_x, recoFid_y, reco_x, reco_y, weights_reco=reco_w, nbx=nbx, nby=nby, 
        nxg=nx_gen, nyg=ny_gen, nxr=nx_rec, nyr=ny_rec, min_gen=10, reg_acc=1e-5, reg_reco=1e-5 )

    # ======================================================
    # phi 
    # ======================================================
    phi_gen_all = eff_Gen_q2["gen_phi"].values
    phi_gen_fid = eff_GenFtr_q2["gen_phi"].values
    phi_reco_fid = eff_RecoGenFtr_q2["Phi_best"].values
    phi_reco = eff_RecoFtr_q2["Phi"].values
    
    centers_phi, acc_phi, acc_phi_model, coef_acc_phi, eff_reco_phi, eff_reco_phi_model, coef_reco_phi, mask_phi = build_efficiency_1d( 
        phi_gen_all, phi_gen_fid, phi_reco_fid, phi_reco, 
        weights_reco=reco_w,nbins=20, n_poly=4, reg_acc=1e-5, reg_reco=1e-5)

    plt.ioff()
    acc_model_file = f"acc_gen_model_{binN}.json"
    reco_model_file = f"eff_reco_model_{binN}.json"
    path_models = f"models/{binN}/"
    path_plots = f"plots/projections/{binN}/"

    save_bernstein2d_model(path_models + reco_model_file, coef_reco, nx_rec, ny_rec)
    save_bernstein2d_model(path_models + acc_model_file, coef_acc, nx_gen, ny_gen)
    save_bernstein1d_model(f"models/{binN}/acc_gen_model_phi_{binN}.json", coef_acc_phi, 4)
    save_bernstein1d_model(f"models/{binN}/eff_reco_model_phi_{binN}.json", coef_reco_phi, 4)
    
    # ======================================================
    # PLOTs
    # ======================================================

    plot_projection_x_with_errors(xcenters, acc_gen, acc_gen_model, mask_gen, f"{binN} "+r"Gen Acceptance: $\cos\theta_\ell$", ylim=None, path=path_plots)
    plot_projection_y_with_errors( ycenters, acc_gen, acc_gen_model, mask_gen,  f"{binN} "+r"Gen Acceptance: $\cos\theta_K$",  ylim=None, path=path_plots)
    
    plot_projection_x_with_errors(xcenters, eff_reco, eff_reco_model, mask_gen, f"{binN} "+r"Reco Efficiency: $\cos\theta_\ell$", ylim=None, path=path_plots)
    plot_projection_y_with_errors(ycenters, eff_reco, eff_reco_model, mask_gen, f"{binN} "+r"Reco Efficiency: $\cos\theta_K$", ylim=None, path=path_plots)
    
    plot_projection_x_with_errors( xcenters, acc_gen*eff_reco, acc_gen_model*eff_reco_model, mask_gen, f"{binN} "+r"Total efficiency: projection cos$\theta_\ell$",ylim=None,path=path_plots)
    plot_projection_y_with_errors( xcenters, acc_gen*eff_reco, acc_gen_model*eff_reco_model, mask_gen, f"{binN} "+r"Total efficiency: projection cos$\theta_K$",ylim=None, path=path_plots)

    # Phi
    plot_1d_result(centers_phi, acc_phi, acc_phi_model, mask_phi, f"{binN} "+r"Gen Acceptance $\phi$",ylim=None,path=path_plots)
    plot_1d_result(centers_phi, eff_reco_phi, eff_reco_phi_model, mask_phi, f"{binN} "+r"Reco Efficiency $\phi$",ylim=None,path=path_plots)
    plot_1d_result(centers_phi, acc_phi*eff_reco_phi, acc_phi_model*eff_reco_phi_model, mask_phi, f"{binN} "+r"Total efficiency $\phi$",ylim=None,path=path_plots)
    plt.ion()


In [ ]:
# plt.figure(figsize=(5,5))
# plt.scatter(eff_reco[mask_gen], eff_reco_model[mask_gen], s=8, alpha=0.4)
# plt.plot([0,0.2], [0,0.2], "r--")
# plt.xlabel("Binned efficiency")
# plt.ylabel("Bernstein model")
# plt.grid(True)
# plt.title("Bin-by-bin comparison")
# plt.show()

# Gen Fit Physical space SLSQP

# Gen Fit Physical space 

In [6]:

q2_bins = {"bin1":[1.1, 2.0],"bin2": [2.0, 4.0],"bin3":[4.0, 6.0], "bin4":[6.0, 7.0], "bin5":[7.0, 8.0],"bin7":[11.0, 12.5], "bin9":[15.0, 17.0], "bin10":[17.0, 23.0]}

for binN in q2_bins.keys():
    print(f"\n{'='*60}\nProcesando {binN} con rango q2: {q2_bins[binN]}\n{'='*60}")

    # ======================================================
    # CONFIGURACIÓN DEL ESPACIO 
    # ======================================================
    cos_l = zfit.Space('cos_l', limits=(-1, 1))
    cos_k = zfit.Space('cos_k', limits=(-1, 1))
    phi   = zfit.Space('phi',   limits=(-np.pi, np.pi)) 
    obs_ang = cos_l * cos_k * phi  


    # Parámetros Físicos con límites
    FL  = zfit.Parameter(f'FL_{binN}',  0.5)
    S3  = zfit.Parameter(f'S3_{binN}',  0.0)
    S9  = zfit.Parameter(f'S9_{binN}',  0.0)
    AFB = zfit.Parameter(f'AFB_{binN}', 0.0)
    S4  = zfit.Parameter(f'S4_{binN}',  0.0)
    S7  = zfit.Parameter(f'S7_{binN}',  0.0)
    S5  = zfit.Parameter(f'S5_{binN}',  0.0)
    S8  = zfit.Parameter(f'S8_{binN}',  0.0)
    # Listas auxiliares
    r_keys = ['FL', 'S3', 'S9', 'AFB', 'S4', 'S7', 'S5', 'S8']
    fit_params_list_phy = [FL, S3, S9, AFB, S4, S7, S5, S8]

    # ======================================================
    # CONSTRUCCIÓN DE PDFs Y CARGA DE DATOS
    # ======================================================

    # PDF  
    pdf_ang_phy = FullAngular_Physical_PDF(obs_ang, FL, S3, S9, AFB, S4, S7, S5, S8)
    # Carga de Datos
    obs_Gen_q2 = select_q2_bin(obs_Gen, binN, "q2Gen")
    data_true = zfit.Data.from_numpy(array=obs_Gen_q2[["gen_cosThetaL", "gen_cosThetaK", "gen_phi"]].to_numpy(), obs=obs_ang)

    # ======================================================
    # FITS
    # ======================================================

    print("\n" + "="*60)
    print(">>> INICIANDO FIT GEN LEVEL PHYSICAL PDF <<<")
    print("="*60)
    result_gen_phy, errors_gen_phy = run_fit(pdf_ang_phy, data_true)
    print(result_gen_phy.params)
    # results_gen_save = save_fit_results(result_gen, binN, base_dir="fit_results/gen", name=f"fit_results_gen_transformed_{binN}")
    phy_values = [result_gen_phy.params[p]['value'] for p in fit_params_list_phy]
    print("\n" + "-"*60)
    save_correlation_matrix(result_gen_phy, fit_params_list_phy, binN, out_dir=f"fit_results/gen_phy/{binN}")
    results_gen_save = save_fit_results(result_gen_phy, binN, base_dir="fit_results/gen_phy", name=f"fit_results_gen_physical_{binN}")


    


Procesando bin1 con rango q2: [1.1, 2.0]

>>> INICIANDO FIT GEN LEVEL PHYSICAL PDF <<<


2026-02-21 11:39:16.859156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-21 11:39:17.425237: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-21 11:39:17.426162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

name        value  (rounded)                minos    at limit
--------  ------------------  -------------------  ----------
FL_bin1              0.65282  -  0.016   +  0.015       False
S3_bin1          0.000734259  -  0.021   +  0.021       False
S9_bin1          -0.00751069  -  0.021   +   0.02       False
AFB_bin1           0.0192128  -  0.015   +  0.015       False
S4_bin1            0.0194374  -  0.026   +  0.026       False
S7_bin1           -0.0209414  -  0.024   +  0.025       False
S5_bin1           -0.0249206  -  0.025   +  0.025       False
S8_bin1            0.0203171  -  0.025   +  0.025       False

------------------------------------------------------------
[CheckPoint] Resultados guardados en: fit_results/gen_phy/bin1/fit_results_gen_physical_bin1.json

Procesando bin2 con rango q2: [2.0, 4.0]

>>> INICIANDO FIT GEN LEVEL PHYSICAL PDF <<<
name        value  (rounded)                minos    at limit
--------  ------------------  -------------------  ----------
FL_bin2 

In [ ]:
with open('data_gen_phy_fit.txt', 'w') as archivo:
    archivo.write(data_gen_phy_fit.stdout) 
    archivo.write(data_gen_phy_fit.stderr)


# Gen  FIT

In [7]:
%%capture data_gen_fit

q2_bins = {"bin1":[1.1, 2.0],"bin2": [2.0, 4.0],"bin3":[4.0, 6.0], "bin4":[6.0, 7.0], "bin5":[7.0, 8.0],"bin7":[11.0, 12.5], "bin9":[15.0, 17.0], "bin10":[17.0, 23.0]}

for binN in q2_bins.keys():
    print(f"\n{'='*60}\nProcesando {binN} con rango q2: {q2_bins[binN]}\n{'='*60}")

    # ======================================================
    # CONFIGURACIÓN DEL ESPACIO 
    # ======================================================
    cos_l = zfit.Space('cos_l', limits=(-1, 1))
    cos_k = zfit.Space('cos_k', limits=(-1, 1))
    phi   = zfit.Space('phi',   limits=(-np.pi, np.pi)) 
    obs_ang = cos_l * cos_k * phi  

    lim_val = 150.0 

    # Parámetros Físicos con límites
    rFL  = zfit.Parameter(f'rFL_{binN}',  0.5, lower_limit=-lim_val, upper_limit=lim_val)
    rS3  = zfit.Parameter(f'rS3_{binN}',  0.0, lower_limit=-lim_val, upper_limit=lim_val)
    rS9  = zfit.Parameter(f'rS9_{binN}',  0.0, lower_limit=-lim_val, upper_limit=lim_val)
    rAFB = zfit.Parameter(f'rAFB_{binN}', 0.0, lower_limit=-lim_val, upper_limit=lim_val)
    rS4  = zfit.Parameter(f'rS4_{binN}',  0.0, lower_limit=-0.1, upper_limit=0.1)
    rS7  = zfit.Parameter(f'rS7_{binN}',  0.0, lower_limit=-lim_val, upper_limit=lim_val)
    rS5  = zfit.Parameter(f'rS5_{binN}',  0.0, lower_limit=-lim_val, upper_limit=lim_val)
    rS8  = zfit.Parameter(f'rS8_{binN}',  0.0, lower_limit=-lim_val, upper_limit=lim_val)
    # Listas auxiliares
    r_keys = ['rFL', 'rS3', 'rS9', 'rAFB', 'rS4', 'rS7', 'rS5', 'rS8']
    fit_params_list = [rFL, rS3, rS9, rAFB, rS4, rS7, rS5, rS8]

    # ======================================================
    # CONSTRUCCIÓN DE PDFs Y CARGA DE DATOS
    # ======================================================

    # PDF  Transformada
    pdf_ang_trans = FullAngular_Transformed_PDF(obs_ang, rFL, rS3, rS9, rAFB, rS4, rS7, rS5, rS8)
    # Carga de Datos
    obs_Gen_q2 = select_q2_bin(obs_Gen, binN, "q2Gen")
    data_true = zfit.Data.from_numpy(array=obs_Gen_q2[["gen_cosThetaL", "gen_cosThetaK", "gen_phi"]].to_numpy(), obs=obs_ang)

    # ======================================================
    # FITS
    # ======================================================

    print("\n" + "="*60)
    print(">>> INICIANDO FIT GEN LEVEL ")
    print("="*60)
    result_gen, errors_gen = run_fit(pdf_ang_trans, data_true)
    print(result_gen.params)
    # results_gen_save = save_fit_results(result_gen, binN, base_dir="fit_results/gen", name=f"fit_results_gen_transformed_{binN}")
    r_values = [result_gen.params[p]['value'] for p in fit_params_list]
    phys_vals_gen = apply_transformation_equations(*r_values)
    print("\n" + "-"*60)
    print(f"RESUMEN DE OBSERVABLES FÍSICOS (Bin: {binN})")
    print("-"*60)
    print(f"{'Observable':<10} | {'Valor Físico':<15}")
    print("-"*60)
    print_order = ['FL', 'AFB', 'S3', 'S4', 'S5', 'S7', 'S8', 'S9']    
    for key in print_order:
        val = phys_vals_gen.get(key, 0.0)
        print(f"{key:<10} | {val:>15.6f}")
    print("-"*60 + "\n")

    save_correlation_matrix(result_gen, fit_params_list, binN, out_dir=f"fit_results/gen/{binN}")
    r_values = [result_gen.params[p]['value'] for p in fit_params_list]
    phys_vals_gen = apply_transformation_equations(*r_values)



    

In [8]:
with open('data_gen_fit.txt', 'w') as archivo:
    archivo.write(data_gen_fit.stdout) 
    archivo.write(data_gen_fit.stderr)


# Fit efficiecnasi

In [ ]:
%%capture mi_registro
q2_bins = {"bin1":[1.1, 2.0],"bin2": [2.0, 4.0],"bin3":[4.0, 6.0], "bin4":[6.0, 7.0], "bin5":[7.0, 8.0],"bin7":[11.0, 12.5], "bin9":[15.0, 17.0], "bin10":[17.0, 23.0]}




for binN in q2_bins.keys():
    print(f"\n{'='*60}\nProcesando {binN} con rango q2: {q2_bins[binN]}\n{'='*60}")

    # ======================================================
    # CONFIGURACIÓN DEL ESPACIO 
    # ======================================================
    cos_l = zfit.Space('cos_l', limits=(-1, 1))
    cos_k = zfit.Space('cos_k', limits=(-1, 1))
    phi   = zfit.Space('phi',   limits=(-np.pi, np.pi)) 
    obs_ang = cos_l * cos_k * phi  

    # Parámetros Físicos
    rFL  = zfit.Parameter(f'rFL_{binN}',  0.1, step_size=0.1)
    rS3  = zfit.Parameter(f'rS3_{binN}',  0.0, step_size=0.1)
    rS9  = zfit.Parameter(f'rS9_{binN}',  0.0, step_size=0.1)
    rAFB = zfit.Parameter(f'rAFB_{binN}', 0.0, step_size=0.1)
    rS4  = zfit.Parameter(f'rS4_{binN}',  0.0, step_size=0.1)
    rS7  = zfit.Parameter(f'rS7_{binN}',  0.0, step_size=0.1)
    rS5  = zfit.Parameter(f'rS5_{binN}',  0.0, step_size=0.1)
    rS8  = zfit.Parameter(f'rS8_{binN}',  0.0, step_size=0.1)
    # Listas auxiliares
    r_keys = ['rFL', 'rS3', 'rS9', 'rAFB', 'rS4', 'rS7', 'rS5', 'rS8']
    fit_params_list = [rFL, rS3, rS9, rAFB, rS4, rS7, rS5, rS8]

    # ======================================================
    # CONSTRUCCIÓN DE PDFs Y CARGA DE DATOS
    # ======================================================

    # PDF  Transformada
    pdf_ang_trans = FullAngular_Transformed_PDF(obs_ang, rFL, rS3, rS9, rAFB, rS4, rS7, rS5, rS8)

    # PDF de Eficiencia
    coef_acc, nx_acc, ny_acc = load_bernstein_model(f"models/{binN}/acc_gen_model_{binN}.json")
    coef_acc_phi, n_phi_acc = load_bernstein1d_model(f"models/{binN}/acc_gen_model_phi_{binN}.json")

    coef_reco, nx_reco, ny_reco = load_bernstein_model(f"models/{binN}/eff_reco_model_{binN}.json")
    coef_reco_phi, n_phi_reco = load_bernstein1d_model(f"models/{binN}/eff_reco_model_phi_{binN}.json")

    eff_pdf = Efficiency_Bernstein_Factorized(
        obs=obs_ang, coef_acc_2d=coef_acc, coef_acc_phi=coef_acc_phi, nx_acc=nx_acc, ny_acc=ny_acc, n_phi_acc=n_phi_acc,
        coef_reco_2d=coef_reco, coef_reco_phi=coef_reco_phi,nx_reco=nx_reco, ny_reco=ny_reco, n_phi_reco=n_phi_reco, name=f"Eff_Model_{binN}")
    pdf_sig = zfit.pdf.ProductPDF([pdf_ang_trans, eff_pdf])


    # Carga de Datos
    obs_Gen_q2 = select_q2_bin(obs_Gen, binN, "q2Gen")
    obs_RecoFtr_q2 = select_q2_bin(obs_RecoFtr, binN, "q2")

    data_true = zfit.Data.from_numpy(array=obs_Gen_q2[["gen_cosThetaL", "gen_cosThetaK", "gen_phi"]].to_numpy(), obs=obs_ang)
    data_reco = zfit.Data.from_numpy(array=obs_RecoFtr_q2[["CosThetaL", "CosThetaK", "Phi"]].to_numpy(), obs=obs_ang)
    
    # ======================================================
    # FITS
    # ======================================================

    print("\n" + "="*60)
    print(">>> INICIANDO FIT GEN LEVEL (CONTROL)")
    print("="*60)
    result_gen, errors_gen = run_fit(pdf_ang_trans, data_true)
    print(result_gen.params)
    results_gen_save = save_fit_results(result_gen, binN, base_dir="fit_results/gen", name=f"fit_results_gen_transformed_{binN}")
    r_values = [result_gen.params[p]['value'] for p in fit_params_list]
    phys_vals_gen = apply_transformation_equations(*r_values)
    print("\n" + "-"*60)
    print(f"RESUMEN DE OBSERVABLES FÍSICOS (Bin: {binN})")
    print("-"*60)
    print(f"{'Observable':<10} | {'Valor Físico':<15}")
    print("-"*60)
    print_order = ['FL', 'AFB', 'S3', 'S4', 'S5', 'S7', 'S8', 'S9']    
    for key in print_order:
        val = phys_vals_gen.get(key, 0.0)
        print(f"{key:<10} | {val:>15.6f}")
    print("-"*60 + "\n")




    # print("\n" + "="*60)
    # print(">>> INICIANDO FIT RECO)")
    # print("="*60)
    # for p in fit_params_list: p.set_value(0.01)    
    # pdf_sig.update_integration_options(max_draws=200000, tol=1e-5)
    # result_reco, errors_reco = run_fit(pdf_sig, data_reco)
    # print("\n" + "="*60)
    # print(">>> RESULTADOS FINALES DEL FIT RECO")
    # print("="*60)
    # print(result_reco)
    # results_save = save_fit_results(result_reco, binN, base_dir="fit_results/reco")

    # # callculo y tabla de Observables Físicos para RECO
    # r_values_reco = [result_reco.params[p]['value'] for p in fit_params_list]
    # phys_vals_reco = apply_transformation_equations(*r_values_reco)
    # print("\n" + "-"*60)
    # print(f"RESUMEN DE OBSERVABLES FÍSICOS RECO (Bin: {binN})")
    # print("-"*60)
    # print(f"{'Observable':<10} | {'Valor Físico':<15}")
    # print("-"*60)
    # print_order = ['FL', 'AFB', 'S3', 'S4', 'S5', 'S7', 'S8', 'S9']    
    # for key in print_order:
    #     val = phys_vals_reco.get(key, 0.0)
    #     print(f"{key:<10} | {val:>15.6f}")
    # print("-"*60 + "\n")


    

In [ ]:
# 1. Guardar en el archivo de texto
with open('registro_celda.txt', 'w') as archivo:
    archivo.write(mi_registro.stdout) # Guarda los prints normales
    archivo.write(mi_registro.stderr) # Guarda los mensajes del sistema/TensorFlow

# 2. (Opcional) Mostrarlo en la pantalla del notebook si también quieres verlo
mi_registro.show()